本教程主要介绍如何部署 Azure 机器学习扩展

## 前置条件

- 在 Azure 中运行的 AKS 群集。 如果以前没有使用过群集扩展，则需要[注册 KubernetesConfiguration 服务提供程序](https://learn.microsoft.com/zh-cn/azure/aks/dapr#register-the-kubernetesconfiguration-resource-provider)。
- 出于生产目的，Kubernetes 群集必须至少具有 4 个 vCPU 核心和 14 GB 内存。 有关资源及群集大小建议的详细信息，请参阅[建议的资源计划](https://learn.microsoft.com/zh-cn/azure/machine-learning/reference-kubernetes?view=azureml-api-2)。
- 在出站代理服务器或防火墙后面运行的群集需要额外的**网络配置**。
- 安装或升级 Azure CLI，确保其版本为 2.24.0 或更高版本。
- 安装或升级 Azure CLI 扩展 `k8s-extension`，确保其版本为 1.2.3 或更高版本。

## 限制

- Azure 机器学习不支持[将服务主体与 AKS 配合使用](https://learn.microsoft.com/zh-cn/azure/aks/kubernetes-service-principal)。 AKS 群集必须改用托管标识。 同时支持系统分配的托管标识和用户分配的托管标识。 有关详细信息，请参阅[在 Azure Kubernetes 服务中使用托管标识](https://learn.microsoft.com/zh-cn/azure/aks/use-managed-identity)。
    - 将 AKS 群集使用的服务主体转换为使用托管标识后，在安装扩展之前，需要删除并重新创建所有节点池，而不是直接更新。
- Azure 机器学习不支持为 AKS [禁用本地帐户](https://learn.microsoft.com/zh-cn/azure/aks/manage-local-accounts-managed-azure-ad#disable-local-accounts)。 部署 AKS 群集时，本地帐户默认处于启用状态。
- 如果 AKS 群集已启用[授权 IP 范围以访问 API 服务器](https://learn.microsoft.com/zh-cn/azure/aks/api-server-authorized-ip-ranges)，请为该 AKS 群集启用 Azure 机器学习控制平面 IP 范围。 Azure 机器学习控制平面跨配对区域部署。 如果没有 API 服务器的访问权限，则无法部署机器学习 Pod。 在 AKS 群集中启用 IP 范围时，请对两个[配对区域](https://www.microsoft.com/en-us/download/details.aspx?id=56519)都使用 [IP 范围](https://learn.microsoft.com/zh-cn/azure/reliability/cross-region-replication-azure)。
- Azure 机器学习不支持跨订阅附加 AKS 群集。 如果 AKS 群集位于其他订阅中，必须先[将其连接到 Azure-Arc](https://learn.microsoft.com/zh-cn/azure/azure-arc/kubernetes/quickstart-connect-cluster)，并在与 Azure 机器学习工作区相同的订阅中指定。
- Azure 机器学习不保证支持 AKS 中的所有预览阶段功能。 例如，不支持 [Microsoft Entra Pod 标识](https://learn.microsoft.com/zh-cn/azure/aks/use-azure-ad-pod-identity)。
- 如果已按照 [Azure 机器学习 AKS v1 文档](https://learn.microsoft.com/zh-cn/azure/machine-learning/how-to-create-attach-kubernetes?view=azureml-api-1&preserve-view=true)中的步骤创建或附加 AKS 作为推理群集，请在继续下一步之前使用以下链接[清理旧版 azureml-fe 相关资源](https://learn.microsoft.com/zh-cn/azure/machine-learning/how-to-create-attach-kubernetes?view=azureml-api-1&preserve-view=true#delete-azureml-fe-related-resources)。

## Azure 机器学习扩展部署 - CLI 示例

若要使用 CLI 部署 Azure 机器学习扩展，请使用 `az k8s-extension create` 命令传入强制参数的值。

下面仅列出了用户快速进行概念证明的扩展部署场景以供参考。 
若要为生产用途部署扩展，请仔细阅读[配置设置](https://learn.microsoft.com/zh-cn/azure/machine-learning/how-to-deploy-kubernetes-extension?view=azureml-api-2&tabs=deploy-extension-with-cli#review-azure-machine-learning-extension-configuration-settings)的完整列表。

对于 AKS 群集上的 Azure 机器学习扩展部署，请确保为 `managedClusters` 参数指定 `--cluster-type` 值。 运行以下 Azure CLI 命令以部署 Azure 机器学习扩展：

In [ ]:
extension_name="text-aks-ex"
cluster_name="text-aks-cluster"
resource_group="text-aks-rg"

In [ ]:
!az k8s-extension create  --name {extension_name} --extension-type Microsoft.AzureML.Kubernetes --config enableTraining=True enableInference=True inferenceRouterServiceType=LoadBalancer allowInsecureConnections=True InferenceRouterHA=False --cluster-type managedClusters --cluster-name {cluster_name} --resource-group {resource_group} --scope cluster

- `--name`: 为扩展指定一个唯一的名称
- `--extension-type`: 指定要安装的扩展类型，这里是 Azure Machine Learning 的 Kubernetes 扩展

**配置参数**：
```bash
--config enableTraining=True \         # 启用训练功能
        enableInference=True \         # 启用推理功能
        inferenceRouterServiceType=LoadBalancer \  # 推理路由器服务类型
        allowInsecureConnections=True \  # 允许不安全连接
        InferenceRouterHA=False        # 禁用推理路由器高可用性
```
- `enableTraining`: 是否启用机器学习训练功能
- `enableInference`: 是否启用机器学习推理功能
- `inferenceRouterServiceType`: 设置推理路由器的服务类型为 LoadBalancer
- `allowInsecureConnections`: 是否允许不安全连接（生产环境建议设置为 False）
- `InferenceRouterHA`: 是否启用推理路由器的高可用性配置

**集群相关参数**：

- `--cluster-type`: 指定集群类型，这里是托管的 AKS 集群
- `--cluster-name`: 要安装扩展的 AKS 集群名称
- `--resource-group`: 集群所在的资源组
- `--scope`: 扩展的安装范围，这里是整个集群级别


### 验证 Azure 机器学习扩展部署

运行以下 CLI 命令来检查 Azure 机器学习扩展详细信息：

In [ ]:
!az k8s-extension show --name {extension_name} --cluster-type connectedClusters --cluster-name {cluster_name} --resource-group {resource_group}

- `--name {extension_name}`: 
  - 要查询的扩展的名称
  - 必须与创建时指定的扩展名称相匹配
  - 在同一集群中必须是唯一的

**集群类型参数**：
- `--cluster-type connectedClusters`:
  - 指定集群的类型
  - `connectedClusters` 表示这是一个通过 Azure Arc 连接的集群
  - 这与之前创建命令中的 `managedClusters` 不同，表明这是用于查看 Arc 启用的 Kubernetes 集群的扩展

**集群标识参数**：
- `--cluster-name {cluster_name}`:
  - 目标 Kubernetes 集群的名称
  - 必须是在指定资源组中存在的集群
- `--resource-group {resource_group}`:
  - 包含目标集群的 Azure 资源组名称
  - 用于定位集群所在的资源组

在响应中，查找 "name" 和 "provisioningState": "Succeeded"。 请注意，前几分钟的响应可能显示 "provisioningState": "Pending"。
    
如果 provisioningState 显示 Succeeded，请在将 kubeconfig 文件指向你的群集的情况下，在计算机上运行以下命令，以检查 azureml 命名空间下的所有 pod 是否都处于“Running”状态：

In [ ]:
!kubectl get pods -n azureml

## 查看 Azure 机器学习扩展组件

Azure 机器学习扩展部署完成后，可以使用 `kubectl get deployments -n azureml` 查看群集中创建的资源列表。 其中通常包含每个指定配置设置的以下资源的子集。